In [1]:
from urllib.request import urlretrieve

In [3]:
import os
 
output_relative_dir = '../data/'

# check if path exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# creating paths for each type of data set now, for each type of data set we will need, we will create the paths
for target_dir in ('tlc_data', 'tute_data'): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [4]:
YEAR = '2023'
# adjust the range function to the numerical months i.e 1 = jan, 2 = feb, etc...
# MONTHS = range(1, 13)
MONTHS = range(1, 13)

In [5]:
# this is the URL template as of 07/2023
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_"#year-month.parquet

In [6]:
# data output directory is `data/tlc_data/`
tlc_output_dir = output_relative_dir + 'tlc_data'


for month in MONTHS:
    # 0-fill i.e 1 -> 01, 2 -> 02, etc
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    
    # generate url
    url = f'{URL_TEMPLATE}{YEAR}-{month}.parquet'
    # generate output location and filename
    output_dir = f"{tlc_output_dir}/{YEAR}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04
Begin month 05
Completed month 05
Begin month 06
Completed month 06
Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


## Inspecting Data

In [19]:
import pandas as pd

df = pd.read_parquet('../data/tlc_data/2023-01.parquet')
df.tail()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
18479026,HV0003,B03404,B03404,2023-01-31 23:33:42,2023-01-31 23:37:04,2023-01-31 23:39:34,2023-01-31 23:45:11,161,100,0.67,...,1.31,2.75,0.0,0.0,10.62,N,N,,N,N
18479027,HV0003,B03404,B03404,2023-01-31 23:31:20,2023-01-31 23:35:17,2023-01-31 23:35:25,2023-01-31 23:47:28,129,56,2.04,...,0.88,0.00,0.0,0.0,9.82,N,N,,N,N
18479028,HV0003,B03404,B03404,2023-01-31 23:15:39,2023-01-31 23:16:35,2023-01-31 23:18:09,2023-01-31 23:34:46,234,236,4.14,...,3.41,2.75,0.0,0.0,24.22,N,N,,N,N
18479029,HV0003,B03404,B03404,2023-01-31 23:40:03,2023-01-31 23:41:31,2023-01-31 23:44:07,2023-01-31 23:52:16,237,163,1.10,...,1.37,2.75,0.0,0.0,12.62,N,N,,N,N
18479030,HV0003,B03404,B03404,2023-01-31 23:53:42,2023-01-31 23:55:27,2023-01-31 23:56:17,2023-02-01 00:04:59,161,113,2.03,...,1.91,2.75,0.0,0.0,14.88,N,N,,N,N


In [18]:
import base64
import traceback
RED = '\033[91m'
GREEN = '\033[92m'
BOLD = '\033[1m'
RESET = '\033[0m'

try:
    from pyspark.sql import SparkSession
    # Create a spark session (which will run spark jobs)
    spark = (
        SparkSession.builder.appName("MAST30034 Tutorial")
        .config("spark.sql.repl.eagerEval.enabled", True) 
        .config("spark.sql.parquet.cacheMetadata", "true")
        .config("spark.sql.session.timeZone", "Etc/UTC")
        .getOrCreate()
    )
    print(f"{GREEN}{BOLD}Success! Your environment is set up and you are ready for the first workshop.{RESET}")
except Exception as e:
    print(f"{RED}{BOLD}Something went wrong. Reinstall and try again.{RESET}")
    traceback.print_exc()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 19:59:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Success! Your environment is set up and you are ready for the first workshop.


24/08/07 19:59:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [24]:
sdf_jan = spark.read.parquet('../data/tlc_data/2023-01.parquet')
sdf_jan.show(1, vertical=True, truncate=100)

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2023-01-01 00:18:06 
 on_scene_datetime    | 2023-01-01 00:19:24 
 pickup_datetime      | 2023-01-01 00:19:38 
 dropoff_datetime     | 2023-01-01 00:48:07 
 PULocationID         | 48                  
 DOLocationID         | 68                  
 trip_miles           | 0.94                
 trip_time            | 1709                
 base_passenger_fare  | 25.95               
 tolls                | 0.0                 
 bcf                  | 0.78                
 sales_tax            | 2.3                 
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 5.22                
 driver_pay           | 27.83               
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

In [26]:
sdf_jan.limit(5)

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0003,B03404,B03404,2023-01-01 00:18:06,2023-01-01 00:19:24,2023-01-01 00:19:38,2023-01-01 00:48:07,48,68,0.94,1709,25.95,0.0,0.78,2.3,2.75,0.0,5.22,27.83,N,N,,N,N
HV0003,B03404,B03404,2023-01-01 00:48:42,2023-01-01 00:56:20,2023-01-01 00:58:39,2023-01-01 01:33:08,246,163,2.78,2069,60.14,0.0,1.8,5.34,2.75,0.0,0.0,50.15,N,N,,N,N
HV0003,B03404,B03404,2023-01-01 00:15:35,2023-01-01 00:20:14,2023-01-01 00:20:27,2023-01-01 00:37:54,9,129,8.81,1047,24.37,0.0,0.73,2.16,0.0,0.0,0.0,20.22,N,N,,N,N
HV0003,B03404,B03404,2023-01-01 00:35:24,2023-01-01 00:39:30,2023-01-01 00:41:05,2023-01-01 00:48:16,129,129,0.67,431,13.8,0.0,0.41,1.22,0.0,0.0,0.0,7.9,N,N,,N,N
HV0003,B03404,B03404,2023-01-01 00:43:15,2023-01-01 00:51:10,2023-01-01 00:52:47,2023-01-01 01:04:51,129,92,4.38,724,20.49,0.0,0.61,1.82,0.0,0.0,0.0,16.48,N,N,,N,N


In [27]:
sdf_all = spark.read.parquet('../data/tlc_data')

In [29]:
sdf_jan.count(), sdf_all.count()

(18479031, 232490020)

In [30]:
sdf_jan.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

24/08/07 23:40:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 218543 ms exceeds timeout 120000 ms
24/08/07 23:40:05 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/07 23:40:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# Extract the date as a standalone feature
df = df.withColumn("pickup_date", 
                   func.to_date(func.col("pickup_datetime")))